<a href="https://colab.research.google.com/github/srikarraju/GridWorld/blob/main/Natural_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import gym
from collections import deque
import numpy as np

In [ ]:
class policy_net(nn.Module):
  def __init__(self,state_dim,hidden_dim,action_dim):
    super(policy_net,self).__init__()
    self.h = nn.Linear(state_dim,hidden_dim)
    self.out = nn.Linear(hidden_dim,action_dim)

  def forward(self,x):
    x = F.relu(self.h(x))
    x = F.softmax(self.out(x),dim=1)
    return x

class value_net(nn.Module):
  def __init__(self,state_dim,hidden_dim,action_dim):
    super(value_net,self).__init__()
    self.h = nn.Linear(state_dim,hidden_dim)
    self.out = nn.Linear(hidden_dim,1)

  def forward(self,x):
    x = F.relu(self.h(x))
    x = self.out(x)
    return x

In [ ]:
env = gym.make('CartPole-v0')
policy = policy_net(env.observation_space.shape[0],100,env.action_space.n)
value_fn = value_net(env.observation_space.shape[0],100,env.action_space.n)
optimizer1 = torch.optim.Adam(policy.parameters())
optimizer2 = torch.optim.Adam(value_fn.parameters())
gamma = 0.99
alpha = 0.01
returns = deque(maxlen=100)

In [ ]:
n_episode = 1
returns,losses = [], []
while n_episode < 5000:
  state = env.reset()
  rewards, states, actions = [], [], []
  while True:
    probs = policy(torch.tensor(state).unsqueeze(0).float())
    sampler = Categorical(probs)
    action = sampler.sample()

    new_state, reward, done, info = env.step(action.item())

    states.append(state)
    actions.append(action)
    rewards.append(reward)

    state = new_state
    if done == True:
      break

  rewards = np.array(rewards)
  R = torch.tensor([np.sum(rewards[i:]*(gamma**np.array(range(0,len(rewards)-i)))) for i in range(len(rewards))]).float()
  states = torch.tensor(states).float()
  actions = torch.tensor(actions)
  state_values = torch.flatten(value_fn(states).float())

  probs = policy(states)
  sampler = Categorical(probs)
  logprobs = -sampler.log_prob(actions)

  policy_net_loss = torch.sum(logprobs)
  #print(policy_net_loss)
  optimizer1.zero_grad()
  policy_net_loss.backward()
  optimizer1.step()

  returns.append(np.sum(rewards))
  if n_episode%100==0:
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.mean(returns)))
  returns.append(np.mean(returns))
  n_episode += 1

env.close()




Episode:    100	Avg. Return:  22.65
Episode:    200	Avg. Return:  22.13
Episode:    300	Avg. Return:  21.39
Episode:    400	Avg. Return:  21.27
Episode:    500	Avg. Return:  22.05
Episode:    600	Avg. Return:  23.15
Episode:    700	Avg. Return:  24.02
Episode:    800	Avg. Return:  24.66
Episode:    900	Avg. Return:  25.12
Episode:   1000	Avg. Return:  24.71
Episode:   1100	Avg. Return:  24.20
Episode:   1200	Avg. Return:  23.82
Episode:   1300	Avg. Return:  23.47
Episode:   1400	Avg. Return:  23.12
Episode:   1500	Avg. Return:  22.83
Episode:   1600	Avg. Return:  22.62
Episode:   1700	Avg. Return:  22.68
Episode:   1800	Avg. Return:  22.94
Episode:   1900	Avg. Return:  23.40
Episode:   2000	Avg. Return:  23.93
Episode:   2100	Avg. Return:  24.34
Episode:   2200	Avg. Return:  24.54
Episode:   2300	Avg. Return:  24.59
Episode:   2400	Avg. Return:  24.63
Episode:   2500	Avg. Return:  24.66
Episode:   2600	Avg. Return:  24.88
Episode:   2700	Avg. Return:  24.89
Episode:   2800	Avg. Return:

KeyboardInterrupt: ignored